In [1]:
import geopandas as gpd
import pandas as pd
from utils import gdf_operations, functions

2025-01-23 14:52:09.238 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
distrito =gdf_operations.get_dados('distrito')
subbac = gdf_operations.get_dados('subbac')
subpref = gdf_operations.get_dados('subpref')
fcu = gdf_operations.get_dados('fcu')

In [3]:
cd_subbac = gdf_operations.find_subbac_name(subbac, 'cd_')
nm_subbac = gdf_operations.find_subbac_name(subbac, 'nm_')

unidades_list = [
    ('subbac', "Sub Bacias Hidrográficas", cd_subbac, nm_subbac),
    ('subpref', "Subprefeituras", "cd_subpref" , 'nm_subpref'),
    ('distrito', "Distritos", 'cd_distrit', 'nm_distrit'),
    ('fcu', "Favelas e Comunidades Urbanas", 'cd_fcu', 'nm_fcu')
    
]
unidades_df = pd.DataFrame(unidades_list, columns=['gdf_name', 'name', 'column_cd', 'column_name'])


Vou começar pelo fcu para ver se dá certo.

# FCU

In [4]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:118: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [5]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [6]:
overlay_gdf = gdf_operations.overlay_intersec('distrito', gdf_unidade, 'cd_fcu')

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:107: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Distritos

Agora vamos tentar com os distritos

In [7]:
distrito_sorted = gdf_operations.create_gdf_sorted(
    distrito, 'distrito'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:118: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [8]:
choice_unidade = 'distrito'
gdf_unidade = distrito_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [9]:
overlay_gdf = gdf_operations.overlay_intersec(
    'subpref', gdf_unidade, 'cd_distrit')

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:107: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


# Loop por intersec

## Vamos Começar com o FCU mesmo

In [10]:
fcu_sorted = gdf_operations.create_gdf_sorted(
    fcu, 'fcu'
)

C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:118: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted[column] = gdf[column]


In [11]:
choice_unidade = 'fcu'
gdf_unidade = fcu_sorted
name_column_unidade = functions.find_gdf_info(unidades_df, choice_unidade, 'gdf_name', 'column_name')

In [12]:
print(unidades_df[unidades_df['gdf_name']=='fcu'])

  gdf_name                           name column_cd column_name
3      fcu  Favelas e Comunidades Urbanas    cd_fcu      nm_fcu


In [13]:
index_unidade = (
        unidades_df[
            unidades_df['gdf_name'] == choice_unidade
        ].index[0]
    )

name_gdf_unidade = choice_unidade
batatinha = pd.DataFrame()

cd_unidade = gdf_operations.find_gdf_name(gdf_unidade, choice_unidade, 'cd_')

overlay_gdf = gpd.GeoDataFrame()

for i, row in unidades_df.iterrows():
        if i< index_unidade:
            name_gdf_intersec = row.loc['gdf_name']
            cd_intersec = row.loc['column_cd']
            overlay_gdf = gdf_operations.overlay_intersec(
                name_gdf_intersec= name_gdf_intersec, 
                gdf_unidade = gdf_unidade, 
                name_gdf_unidade=name_gdf_unidade
            )
            print(overlay_gdf.columns)
            
            gdf_unidade[cd_unidade] = gdf_unidade[cd_unidade].astype(str)
            overlay_gdf[cd_intersec] = overlay_gdf[cd_intersec].astype(int).astype(str)
            overlay_gdf[cd_unidade] = overlay_gdf[cd_unidade].astype(str)


            gdf_unidade = pd.merge(gdf_unidade, overlay_gdf[[cd_unidade, cd_intersec]], on=cd_unidade, how='left')


gdf_operations.save_intersec(gdf_unidade, choice_unidade)


           
            




C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:107: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'nm_bacia_h', 'geometry', 'area'],
      dtype='object')


C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:107: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'nm_subpref', 'geometry', 'area'],
      dtype='object')


C:\Users\x526378\Desktop\projetos\saneamento\hydra\streamlit-pmsb\utils\gdf_operations.py:107: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  gdf_sorted['geometry'] = gdf['geometry']


Index(['cd_fcu', 'nm_fcu', 'pop_fcu', 'dom_fcu', 'con_fcu', 'cd_identif',
       'cd_subpref', 'cd_distrit', 'nm_distrit', 'geometry', 'area'],
      dtype='object')


PermissionError: [WinError 5] Acesso negado: 'data\\2024_11_26\\intersecs\\fcu'

In [ ]:
subbac.columns

In [ ]:
overlay_gdf = gdf_operations.overlay_intersec( 
    'distrito', 
    gdf_unidade, 
    'fcu'
)

In [ ]:

def intersec_unidades(
    unidades_df,
    choice_unidade,
    gdf_unidade,
    name_column_unidade
)-> gpd.GeoDataFrame:

    gdf_unidade = create_gdf_sorted(gdf=gdf_unidade, name_gdf=choice_unidade)
    index_unidade = (
        unidades_df[
            unidades_df['name'] == choice_unidade
        ].index[0]
    )
    resultados = []
    gdf_final_unidade = gpd.GeoDataFrame()
    
    for i, row in unidades_df.iterrows():
        if i< index_unidade:
            polenta = overlay_intersec(unidades_df, choice_unidade, gdf_unidade, name_column_unidade)
            return polenta

